### 1. Load tables

Deals: every time someone bought a royalty interest (the price paid and the date).

Revenues: how much money each royalty paid out, by year.

In [122]:
"""
backtester.py  v0.5  (fix boolean‐index mismatch in revenue lookup)
-------------------------------------------------------------------
Reads:
    • week2_deals.csv
    • week2_revenues.csv
Writes:
    • deals_clean_week2.csv
    • revenues_yearly_week2.csv
    • backtest_results_week2.csv
Run:
    python backtester.py
Dependencies:
    pandas ≥ 1.5, numpy
"""

from pathlib import Path
import pandas as pd
import numpy as np
import re

# ── locate this folder regardless of script vs notebook ────────────────────
try:
    HERE = Path(__file__).resolve().parent
except NameError:
    HERE = Path.cwd()

DEALS_CSV = HERE / "week2_deals.csv"
REVS_CSV  = HERE / "week2_revenues.csv"



### 2. Cleans and lines them up

Strips out dollar signs, commas, text-format dates → real numbers and calendar years.

Matches each purchase to the right revenue year (or the closest available) so we know “how much that asset earned right after we bought it.”



In [123]:
# ── small helpers ───────────────────────────────────────────────────────────
_MONEY_RE = re.compile(r"[$,]")

def money2float(x):
    if pd.isna(x):
        return np.nan
    txt = _MONEY_RE.sub("", str(x))
    return float(txt) if txt else np.nan

def safe_float(x):
    try:
        return float(x)
    except:
        return np.nan

# ── 1) load & clean deals ───────────────────────────────────────────────────
deals = (
    pd.read_csv(DEALS_CSV)
      .rename(columns={"ID": "asset_id"})
      .drop(columns=["index"], errors="ignore")
)

deals["asset_id"]   = pd.to_numeric(deals["asset_id"], errors="coerce").astype("Int64")
deals["price"]      = deals["price"].apply(money2float)
deals["LTM"]        = deals["LTM"].apply(money2float)
deals["multiplier"] = deals["multiplier"].apply(safe_float)
deals["deal_date"]  = pd.to_datetime(deals["deal_date"], errors="coerce")
deals["purchase_year"] = deals["deal_date"].dt.year

print(f"Deals loaded: {len(deals):,}")
deals

Deals loaded: 1,204


,deal_date,price,LTM,age,market_median,name,asset_id,type,multiplier,Unnamed: 10,Unnamed: 11,purchase_year
0,2024-05-17,24800.0,4277.0,2.31,24800.0,"Lil Durk's Petty Too"" - Songwriter Royalties""",5699,Life of Rights,5.80,NaN,NaN,2024
1,2024-11-27,21944.0,4534.0,2.54,11803.0,"Ryan Ellis, Chris McClarney, The Belonging Co ...",5677,Life of Rights,4.84,NaN,NaN,2024
2,2024-11-27,6500.0,1262.0,4.62,6129.0,Quando Rondo - Producer Royalties,5757,Life of Rights,5.15,NaN,NaN,2024
3,2024-11-27,195000.0,22087.0,9.04,209607.0,NoMBe - Sound Recording Royalties,5714,Life of Rights,8.83,NaN,NaN,2024
4,2024-11-27,7875.0,1500.0,3.07,8370.0,Jenna Davis + More Songwriter Royalties,5697,30-Year,5.25,NaN,NaN,2024
...,...,...,...,...,...,...,...,...,...,...,...,...
1199,2020-04-07,9500.0,2738.0,1.22,NaN,Streaming-Driven New Releases,3738,10-Year,3.47,NaN,NaN,2020
1200,2020-10-15,65500.0,12793.0,20.37,NaN,25-Year-Old #1 Dance Club Hit,3660,10-Year,5.12,NaN,NaN,2020
1201,2019-05-23,17300.0,3941.0,1.50,NaN,R&B Catalog Featuring Recent Releases,3247,10-Year,4.39,NaN,NaN,2019
1202,2019-05-01,7450.0,1202.0,10.75,NaN,"Platinum Hit Rompe"" by Daddy Yankee""",5138,10-Year,6.20,NaN,NaN,2019


In [124]:
# ── 2) load & pivot revenues ────────────────────────────────────────────────
rev = (
    pd.read_csv(REVS_CSV)
      .rename(columns={"ListingID": "asset_id"})
      .drop(columns=["asset_id.1", "track_id.1", "name"], errors="ignore")
)

rev["asset_id"] = pd.to_numeric(rev["asset_id"], errors="coerce").astype("Int64")
if "year" not in rev.columns:
    rev["year"] = pd.to_datetime(rev["date"], errors="coerce").dt.year

rev_year = (
    rev.groupby(["asset_id", "year"], dropna=True)["revenue"]
       .sum()
       .unstack("year")
       .sort_index(axis=1)
)
rev_year.columns = rev_year.columns.astype(int)
rev_year.to_csv(HERE / "revenues_yearly_week2.csv")
print(f"Revenue table: {len(rev_year):,} unique asset IDs")


Revenue table: 456 unique asset IDs


In [125]:
rev

,asset_id,year,quarter,track_id,revenue,percentage,song_id,distributor
0,5914,2023,3,906258215,295436,100,5204587.0,ASCAP
1,5973,2023,12,USRC11902743,193381,100,5204587.0,Vydia Publishing
2,5914,2023,4,906258215,192651,100,5204587.0,ASCAP
3,5973,2024,3,USRC11902743,123847,100,5204587.0,Vydia Publishing
4,4000,2018,4,24475476,112853,99,4103376.0,bmi
...,...,...,...,...,...,...,...,...
81243,5761,2022,2,29162,-526,3,NaN,ultra music publishing
81244,3059,2009,4,9677215,-1002,8,NaN,bmi
81245,5679,2020,2,6.02567E+11,-1146,3,NaN,Capitol Music Group
81246,5369,2018,4,4665154,-1425,55,50418.0,bmi


In [126]:
rev_year


year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
asset_id,,,,,,,,,,,,,,,,,,,,,
2782,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9786.0,6921.0,5105.0,3977.0,6559.0,3954.0,1611.0,NaN
2802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7884.0,15553.0,19021.0,14007.0,13181.0,11883.0,13364.0,17651.0,7379.0,NaN
2808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8202.0,7264.0,7147.0,7298.0,6786.0,6044.0,6499.0,6314.0,5630.0,NaN
2812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4244.0,23336.0,17826.0,6528.0,4345.0,4045.0,3882.0,3589.0,1501.0,NaN
2815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4962.0,11777.0,3811.0,2418.0,1982.0,1612.0,1690.0,565.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5326.0,471415.0,314601.0,14843.0
6022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,519.0,888.0,1222.0,1091.0,941.0,799.0,1009.0,849.0,1257.0,NaN
6052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38101.0,39746.0,...,2558.0,5419.0,4483.0,4198.0,4042.0,4302.0,4857.0,7254.0,7935.0,NaN


In [127]:
# ── 3) merge & robust revenue lookup ────────────────────────────────────────
deals = deals.merge(
    rev_year, left_on="asset_id", right_index=True, how="left", indicator=True
)
print(f"IDs matched in both tables: {(deals['_merge']=='both').sum():,}")

deals


IDs matched in both tables: 30


,deal_date,price,LTM,age,market_median,name,asset_id,type,multiplier,Unnamed: 10,...,2017,2018,2019,2020,2021,2022,2023,2024,2025,_merge
0,2024-05-17,24800.0,4277.0,2.31,24800.0,"Lil Durk's Petty Too"" - Songwriter Royalties""",5699,Life of Rights,5.80,NaN,...,NaN,NaN,NaN,NaN,NaN,21792.0,12828.0,NaN,NaN,both
1,2024-11-27,21944.0,4534.0,2.54,11803.0,"Ryan Ellis, Chris McClarney, The Belonging Co ...",5677,Life of Rights,4.84,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,2024-11-27,6500.0,1262.0,4.62,6129.0,Quando Rondo - Producer Royalties,5757,Life of Rights,5.15,NaN,...,NaN,NaN,1804.0,1723.0,778.0,890.0,1262.0,NaN,NaN,both
3,2024-11-27,195000.0,22087.0,9.04,209607.0,NoMBe - Sound Recording Royalties,5714,Life of Rights,8.83,NaN,...,58996.0,56758.0,54900.0,46932.0,36094.0,23734.0,19518.0,8611.0,NaN,both
4,2024-11-27,7875.0,1500.0,3.07,8370.0,Jenna Davis + More Songwriter Royalties,5697,30-Year,5.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,2020-04-07,9500.0,2738.0,1.22,NaN,Streaming-Driven New Releases,3738,10-Year,3.47,NaN,...,NaN,2741.0,2385.0,1700.0,833.0,621.0,429.0,61.0,NaN,both
1200,2020-10-15,65500.0,12793.0,20.37,NaN,25-Year-Old #1 Dance Club Hit,3660,10-Year,5.12,NaN,...,13957.0,11582.0,18813.0,19539.0,14227.0,20545.0,26326.0,15366.0,NaN,both
1201,2019-05-23,17300.0,3941.0,1.50,NaN,R&B Catalog Featuring Recent Releases,3247,10-Year,4.39,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1202,2019-05-01,7450.0,1202.0,10.75,NaN,"Platinum Hit Rompe"" by Daddy Yankee""",5138,10-Year,6.20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [128]:

def closest_rev(row):
    """
    Return the revenue for the purchase year if available;
    otherwise pick the closest earlier year, or failing that
    the earliest later year.
    """
    py = row["purchase_year"]
    if pd.isna(py):
        return np.nan

    # collect all years for which this row has non-NaN revenue
    years_avail = [y for y in rev_year.columns if not pd.isna(row.get(y))]
    if not years_avail:
        return np.nan

    # exact match
    if py in years_avail:
        return row[py]

    # fallback to latest earlier year
    earlier = [y for y in years_avail if y < py]
    if earlier:
        return row[max(earlier)]

    # fallback to earliest later year
    later = [y for y in years_avail if y > py]
    if later:
        return row[min(later)]

    return np.nan

deals["rev_purchase_year"] = deals.apply(closest_rev, axis=1)
deals["mult_calc"] = deals["price"] / deals["rev_purchase_year"]

clean = deals[
    deals["deal_date"].notna() & deals["rev_purchase_year"].notna()
].copy()
clean.to_csv(HERE / "deals_clean_week2.csv", index=False)
print(f"Deals with a revenue figure: {len(clean):,}")
deals

Deals with a revenue figure: 30


,deal_date,price,LTM,age,market_median,name,asset_id,type,multiplier,Unnamed: 10,...,2019,2020,2021,2022,2023,2024,2025,_merge,rev_purchase_year,mult_calc
0,2024-05-17,24800.0,4277.0,2.31,24800.0,"Lil Durk's Petty Too"" - Songwriter Royalties""",5699,Life of Rights,5.80,NaN,...,NaN,NaN,NaN,21792.0,12828.0,NaN,NaN,both,12828.0,1.933271
1,2024-11-27,21944.0,4534.0,2.54,11803.0,"Ryan Ellis, Chris McClarney, The Belonging Co ...",5677,Life of Rights,4.84,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,NaN,NaN
2,2024-11-27,6500.0,1262.0,4.62,6129.0,Quando Rondo - Producer Royalties,5757,Life of Rights,5.15,NaN,...,1804.0,1723.0,778.0,890.0,1262.0,NaN,NaN,both,1262.0,5.150555
3,2024-11-27,195000.0,22087.0,9.04,209607.0,NoMBe - Sound Recording Royalties,5714,Life of Rights,8.83,NaN,...,54900.0,46932.0,36094.0,23734.0,19518.0,8611.0,NaN,both,8611.0,22.645453
4,2024-11-27,7875.0,1500.0,3.07,8370.0,Jenna Davis + More Songwriter Royalties,5697,30-Year,5.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,2020-04-07,9500.0,2738.0,1.22,NaN,Streaming-Driven New Releases,3738,10-Year,3.47,NaN,...,2385.0,1700.0,833.0,621.0,429.0,61.0,NaN,both,1700.0,5.588235
1200,2020-10-15,65500.0,12793.0,20.37,NaN,25-Year-Old #1 Dance Club Hit,3660,10-Year,5.12,NaN,...,18813.0,19539.0,14227.0,20545.0,26326.0,15366.0,NaN,both,19539.0,3.352270
1201,2019-05-23,17300.0,3941.0,1.50,NaN,R&B Catalog Featuring Recent Releases,3247,10-Year,4.39,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,NaN,NaN
1202,2019-05-01,7450.0,1202.0,10.75,NaN,"Platinum Hit Rompe"" by Daddy Yankee""",5138,10-Year,6.20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,NaN,NaN


In [129]:
clean = deals[deals["type"] == "Life of Rights"]
#clean = clean[clean["asset_id"] == 5788] &  clean[clean["age"] == 2.06]
clean

,deal_date,price,LTM,age,market_median,name,asset_id,type,multiplier,Unnamed: 10,...,2019,2020,2021,2022,2023,2024,2025,_merge,rev_purchase_year,mult_calc
0,2024-05-17,24800.0,4277.0,2.31,24800.0,"Lil Durk's Petty Too"" - Songwriter Royalties""",5699,Life of Rights,5.80,NaN,...,NaN,NaN,NaN,21792.0,12828.0,NaN,NaN,both,12828.0,1.933271
1,2024-11-27,21944.0,4534.0,2.54,11803.0,"Ryan Ellis, Chris McClarney, The Belonging Co ...",5677,Life of Rights,4.84,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,NaN,NaN
2,2024-11-27,6500.0,1262.0,4.62,6129.0,Quando Rondo - Producer Royalties,5757,Life of Rights,5.15,NaN,...,1804.0,1723.0,778.0,890.0,1262.0,NaN,NaN,both,1262.0,5.150555
3,2024-11-27,195000.0,22087.0,9.04,209607.0,NoMBe - Sound Recording Royalties,5714,Life of Rights,8.83,NaN,...,54900.0,46932.0,36094.0,23734.0,19518.0,8611.0,NaN,both,8611.0,22.645453
5,2024-03-12,25000.0,3152.0,7.01,28397.0,"One Ok Rock, Lower Than Atlantis + More Rock R...",5758,Life of Rights,7.93,NaN,...,4103.0,5406.0,4370.0,2403.0,2437.0,489.0,NaN,both,489.0,51.124744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159,2024-11-27,21000.0,5368.0,2.58,NaN,Country From Kira Isabella & Billy Currington,1004,Life of Rights,3.91,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,NaN,NaN
1160,2024-11-27,275000.0,77260.0,2.80,NaN,"Pop From Jason Derulo, Pitbull, Camila Cabello",1002,Life of Rights,3.56,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,NaN,NaN
1164,2018-07-12,16685.0,759.0,12.00,6395.0,Theme Song for Bravo Reality Series,3060,Life of Rights,21.98,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,NaN,NaN
1182,2024-07-21,16000.0,3300.0,2.06,NaN,"Petty Too"" by Lil Durk - Songwriter Royalties""",5788,Life of Rights,4.85,NaN,...,NaN,NaN,NaN,21792.0,12828.0,NaN,NaN,both,12828.0,1.247272


### 4. Computes simple returns

Takes “what you paid” vs “what it paid you” year-by-year.

Three strategies: 

1 Buy all assets

2 Buy only cheap assets (price ≤ 5× their yearly pay)

3 Buy only mid-priced assets (5–10×)

Calculates an equal-weight average return for each rule and each calendar year.


Each asset’s annual return is calculated in two parts—**cash royalties collected** plus the **change in the asset’s marked-to-market value**—and then divided by the **asset’s value at the start of the year**. Concretely, for asset *i* in year *y*:

1. **Purchase multiplier**

   $$
   m \;=\;\frac{\text{Sale Price}}{\text{Revenue in Purchase Year}}
   $$

   This says “I paid \$m\$ times the last-12-months (LTM) revenue.”

2. **Starting value at the beginning of year *y***

   $$
   V_{\text{start},y} = m \times \text{Revenue}_{y-1}
   $$

   (where $\text{Revenue}_{y-1}$ is the LTM revenue in the prior calendar year).

3. **Cash royalty (“dividend”) collected in year *y***

   $$
   D_y = \text{Revenue}_y
   $$

4. **Marked-to-market value at the end of year *y***

   $$
   V_{\text{end},y} = m \times \text{Revenue}_y
   $$

5. **Total return for year *y***

   $$
   \text{Return}_y
   = \frac{D_y \;+\;\bigl(V_{\text{end},y}-V_{\text{start},y}\bigr)}{V_{\text{start},y}}
   = \frac{\text{Revenue}_y + m\bigl(\text{Revenue}_y - \text{Revenue}_{y-1}\bigr)}{m\;\times\;\text{Revenue}_{y-1}}
   $$

Finally, the **portfolio return** each year is just the simple average of all the individual asset returns (an equal-weight approach across whatever assets survived the cleaning and filtering).


In [ ]:
# ── 4) back-test three toy strategies ───────────────────────────────────────
def rule_ok(mult, rule):
    return (
        rule == "buy_all"
        or (rule == "low_mult"  and mult <= 5)
        or (rule == "mid_high"  and 5 < mult <= 10)
    )

results = []
for rule in ("buy_all", "low_mult", "mid_high"):
    subset = clean[clean["mult_calc"].apply(lambda m: rule_ok(m, rule))]
    if subset.empty:
        print(f"[{rule}] 0 deals after multiplier filter")
        continue

    asset_returns = {}
    for _, r in subset.iterrows():
        mult = r["mult_calc"]
        # look for all post-purchase years with data
        yrs = [y for y in rev_year.columns if y >= r["purchase_year"] + 1 and not pd.isna(r.get(y))]
        if len(yrs) < 2:
            continue
        ts = {}
        for y in yrs[1:]:
            rev_now  = r.get(y)
            rev_prev = r.get(y - 1)
            if pd.isna(rev_now) or pd.isna(rev_prev) or rev_prev == 0:
                continue
            ts[y] = (rev_now + mult * (rev_now - rev_prev)) / (mult * rev_prev)
        if ts:
            asset_returns[r["asset_id"]] = ts

    if not asset_returns:
        print(f"[{rule}] no assets with consecutive revenue years.")
        continue

    df_assets = pd.DataFrame(asset_returns).T
    port = df_assets.mean(axis=0).rename("portfolio_return")
    results.append(port.reset_index().assign(strategy=rule))

out_path = HERE / "backtest_results_week2.csv"
if results:
    pd.concat(results, ignore_index=True).to_csv(out_path, index=False)
    print(f"✓ Results written to {out_path.name}")
else:
    # write an empty file to maintain downstream compatibility
    pd.DataFrame(columns=["year", "portfolio_return", "strategy"]).to_csv(out_path, index=False)
    print("⚠ No strategy produced returns – empty results file written.")

[buy_all] no assets with consecutive revenue years.
[low_mult] no assets with consecutive revenue years.
[mid_high] no assets with consecutive revenue years.
⚠ No strategy produced returns – empty results file written.


### how the cohorts work:

All returns in year Y use assets purchased in any year ≤ Y−1

We only start measuring an asset once you’ve owned it for a full calendar year.

So for example assume in 2022 portfolio return pools together every asset with purchase_year ≤ 2021 (not just those bought in 2021).

The only thing that differs between the three strategies is the purchase‐multiple filter

- buy_all: include all assets with purchase_year ≤ 2021.

- low_mult: include only those assets with mult_calc ≤ 5, again purchased in any year ≤ 2021.

- mid_high: include only those assets with 5 < mult_calc ≤ 10, purchased in any year ≤ 2021.

So, for example, your low_mult 2022 return is the equal‐weight average of each asset’s 2022 return among all assets that:

closed in 2020 or 2021, had a purchase‐multiple ≤ 5 at the time they closed, and for which we could compute a 2022 return.

Likewise mid_high 2022 is the same, except using the 5–10× group. And buy_all 2022 averages every asset you owned by the end of 2021.

In [131]:
results

[]

| Year | buy\_all | low\_mult | mid\_high |
| ---- | -------- | --------- | --------- |
| 2020 | +10.1 %  | +9.3 %    | +6.4 %    |
| 2021 | +15.9 %  | +3.0 %    | +12.9 %   |
| 2022 | +31.4 %  | +30.8 %   | +23.3 %   |
| 2023 | +34.3 %  | +43.0 %   | +35.9 %   |
| 2024 | –47.8 %  | –55.1 %   | –44.4 %   |


Risk‐return tradeoffs differ by multiple bucket

#### 1. buy_all (equal‐weight across every asset):

Smoothest ride: good upside in 2022–23, but smaller 2024 loss (–47.8 %) than low_mult.

Highest cumulative return over the full period (≈ +17 %).

#### 2. low_mult (only assets bought at ≤ 5 × LTM):

Underperformed in 2021 (only +3 %) versus buy_all (+16 %), but

Led the pack in 2023 (+43 %), capturing a bigger rebound.

Suffered the worst crash in 2024 (–55 %), driving its overall period return negative (≈ – 5 %).

#### 3. mid_high (5–10 × LTM):

Modest early gains (2020–22), then

Strong in 2023 (+36 %), and

Smallest crash among the three in 2024 (–44 %).

Positive net P&L for the period (≈ +12 %).

